In [7]:
import numpy as np
import pandas as pd
from groq import Groq
import os
import time

In [8]:
df = pd.read_excel('../../data/raw/OMIEC_16_08_24.xls')

In [9]:
df['Abstract']

0      Owing to their outstanding electrical/electroc...
1      A synthesis method for large-scale conjugated ...
2      Understanding the structural and dynamic prope...
3      An essential part of developing organic mixed ...
4      n-Type organic electrochemical transistors (OE...
                             ...                        
631    Achieving high stability is critical for the i...
632    Cytochrome P-450 2D is a subfamily of the cyto...
633    Glucocorticoids are being found to influence e...
634    The identification of P450 mRNAs in a tissue p...
635    Growth hormone (GH) may directly affect the ex...
Name: Abstract, Length: 636, dtype: object

In [10]:
GROQ_API_KEY = 'GROQ_API_KEY'

In [15]:
# groq
client = Groq(
    api_key=GROQ_API_KEY,
)

# respostas
responses_list = []

# todos os abstracts
total = len(df['Abstract'])
total_time = 0


for index, row in df.iterrows():
    start_time = time.time()

    # tempo
    print('\n', index, 'of', total, 'remaining estimated time', (total-index)*(total_time/(index+1)))

    # chat request
    completion = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
            {
                "role": "system",
                "content": "RolePlay as a bot seeking for polymers of the type OMIEC, also known as Organic Mixed Ionic Electronic Conductors."
            },
            {
                "role": "user",
                "content": "For the text I will input next, provide only the names of OMIEC polymers, also known as Organic Mixed Ionic Electronic Conductors, that are present in the text. If any polymer that meets the condition is found in the text, the output should be only the polymer material names separated by semicolons. If no polymer that meets the condition is found in the text, the output should be 'None'. Do not repeat the polymer name more than once. The output should not include the type of material. I want to use the output for future data analysis, so if the polymer name is written in full, replace it with its abbreviation."
            },
            {
                "role": "assistant",
                "content": "Hello, I will tell you the names of OMIEC-type polymers mentioned in any text you input."
            },
            {
                "role": "user",
                "content": f"{row['Abstract']}"  # Use f-string to format the abstract
            }
        ],
        temperature=0.2,
        max_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )

    # coletar respostas
    response = ""

    # Stream the completion result and print the content
    for chunk in completion:
        response += chunk.choices[0].delta.content or ""
        print(chunk.choices[0].delta.content or "", end="")

    print()
    
    # listas de respostas
    responses_list.append({ 'index': index, 'response': response })

    # tempo total
    total_time += time.time() - start_time
    


 0 of 636 remaining estimated time 0.0
None

 1 of 636 remaining estimated time 107.30305850505829
None

 2 of 636 remaining estimated time 134.52905972798666
None

 3 of 636 remaining estimated time 828.8270101547241
None

 4 of 636 remaining estimated time 992.7772846221924
None

 5 of 636 remaining estimated time 1304.7704700231552
PT; ND

 6 of 636 remaining estimated time 1521.932988166809
Mel; PEDOT:PSS

 7 of 636 remaining estimated time 1769.1310528814793
PEDOT:PSS

 8 of 636 remaining estimated time 1954.3845708635117
PEDOT:PSS

 9 of 636 remaining estimated time 2040.3726104021073
None

 10 of 636 remaining estimated time 2140.930888739499
PProDOT

 11 of 636 remaining estimated time 2142.8526068727174
None

 12 of 636 remaining estimated time 2246.4970779418945
PEDOT;

 13 of 636 remaining estimated time 2286.2531735897064
None

 14 of 636 remaining estimated time 2317.0824583053586
None

 15 of 636 remaining estimated time 2308.561402603984
P3MEEET

 16 of 636 remaining es

In [16]:
# DataFrame
df_with_responses = pd.DataFrame(responses_list)

# salvar para .csv
df_with_responses.to_excel('../../dataframes/llama3-70B-OMIEC160824_RESPONSES.xlsx.xlsx', index=False)

#### Playground

In [15]:
df_with_responses

,index,response
0,0,PEDOT
1,1,None
2,2,PEDOT:PolyDADMA; PEDOT:PolyDADMA FSI; PEDOT:Po...
3,3,conducting/semiconducting conjugated polymers
4,4,PEDOT:PSS
...,...,...
870,870,None
871,871,None
872,872,None
873,873,None


In [16]:
result = pd.concat([df_with_responses, df], axis=1)
result.drop('index', axis=1, inplace=True)
result = result[result['response'] != 'None']
result = result.reset_index(drop=True)

In [220]:
result.to_excel('1_OMIEC_RESPONSES_output.xlsx', index=False)

In [214]:
polímeros = []
for response in result['response']:
    polímeros = polímeros + response.split(';')

In [215]:
polímeros_processado = [s.strip() for s in polímeros]

In [226]:
serie_polímeros = pd.Series(polímeros_processado)
polímeros_filtrados = serie_polímeros[~serie_polímeros.str.contains(r'OMIEC.*', regex=True)]
polímeros_filtrados = polímeros_filtrados[polímeros_filtrados != '']
polímeros_filtrados = polímeros_filtrados[~polímeros_filtrados.str.contains(r'(?i)\bnone\b', regex=True)]
polímeros_filtrados = polímeros_filtrados[~polímeros_filtrados.str.contains('conduct')]
polímeros_filtrados = polímeros_filtrados[~polímeros_filtrados.str.contains('conjug')]
polímeros_filtrados = polímeros_filtrados[~polímeros_filtrados.str.contains('cellul')]
polímeros_filtrados = polímeros_filtrados[~polímeros_filtrados.str.contains(r'(?i)\bpolymer\b', regex=True)]
polímeros_filtrados = polímeros_filtrados[~polímeros_filtrados.str.contains(r'(?i)\bmixed\b', regex=True)]

In [248]:
replacements = {
    r'.*\((PEDOT:PSS)\)': r'\1',
    r'.*\((PEDOT)\)': r'\1',
    r'.*\((PEDOT-P)\)': r'\1',
    r'.*\((PEDOT-T)\)': r'\1',
    r'poly\(3,4-ethylenedioxythiophene\)': 'PEDOT',
    r'.*\((P3HT)\)': r'\1',
    r'.*\((PVDF)\)': r'\1',
    r'PEDOT:polystyrenesulfonate': 'PEDOT:PSS',
    r'PEDOT:polystyrene sulfonate': 'PEDOT:PSS',
    r'PEDOT\s*:\s*PSS': 'PEDOT:PSS',
    r'PEDOT\/PSS': 'PEDOT:PSS',
    r'PEDOT\/?poly\(4-styrenesulfonate\)': 'PEDOT:PSS',
    r'PEDOT\s*[-]\s*PSS': 'PEDOT:PSS'


}

In [249]:
for pattern, replacement in replacements.items():
    polímeros_filtrados = polímeros_filtrados.str.replace(pattern, replacement, regex=True, case=False)

In [250]:
len(serie_polímeros)

683

In [251]:
len(polímeros_filtrados.unique())

432

In [252]:
polímeros_filtrados.unique()

array(['PEDOT', 'PEDOT:PolyDADMA', 'PEDOT:PolyDADMA FSI',
       'PEDOT:PolyDADMA TFSI', 'PEDOT:PSS', 'PSS', 'PEO-PEDOT',
       'PANI/PSS', 'poly( vinylferrocene) (PVF)', 'P(3-MT-TTF)', 'PTFE',
       'P(VDF-TrFE)', 'PBTTT', 'PVDF', 'P3HT', 'F4TCNQ', 'BPEEPTF', 'TTF',
       'Bipolar polymers.', 'PEDOT-T', 'PEDOT-P',
       "Poly(3,3'-dialkylimidazolium triflate)",
       'Poly(tetraphenylphosphonium tetracyanometate)',
       'Poly(3-methylthiophene)', 'Poly(o-phenylenediamine)',
       'Poly(3-phenyl-phenylene sulfide)', 'EDOTS', 'EDOTCOOH', 'PProDOT',
       'Polymerized ionic liquid', 'PMDT-TTF', 'TEDFE', 'PT-CSA-O-PTH',
       'PT-TSPT', 'HPTS-PDBT.', 'EG/GOPS-PEDOT:PSS', 'pg2T-TT',
       'f-BTI2g-TVTF', 'f-BTI2g-TVTCl', 'Polyparaphenylene',
       'Ni-3(HITP)(2)', 'PEDOT-Cl', '[C(2)mpyr][FSI]', 'poly-FADS',
       'gNR-Bu', 'PPSS', 'PPTFSI', 'f-BTI2g-SVSCN', 'f-BSeI2g-SVSCN',
       'PyQt', 'PBTTT-C8', 'OMEC-5', '(C16H33CH2CH2CO2)2Sn', 'DTPT',
       'PTAA-DMB', 'OMEM', 'OMEC-3

In [257]:
df_polímeros = pd.DataFrame(polímeros_filtrados, columns=['Polímeros'])

In [258]:
contagem = df_polímeros['Polímeros'].value_counts()

In [259]:
contagem.head(30)

Polímeros
PEDOT:PSS               65
PEDOT                   34
P3HT                    18
TTF-CA                   6
PEO                      6
polythiophene            4
PANI                     4
MEH-PPV                  4
BBL                      3
PSS                      3
PTAA                     3
Polyaniline              3
PPV                      2
Polypyrrole              2
P3MEEET                  2
LiTFSI                   2
CH3NH3PbI3               2
poly(ethylene oxide)     2
Homo-gDPP                2
NDI-T2                   2
PSSNa                    2
Spiro-OMeTAD             2
PPy                      2
g4T2-T2                  2
(BEDT-TTF)               2
PTEO                     2
Polythiophene            2
PTh                      2
p(g2T-T)                 2
P3T                      2
Name: count, dtype: int64

In [260]:
np.unique(df_polímeros[df_polímeros['Polímeros'].str.contains(r'PEDOT', regex=True)].values.tolist())


array(['EG/GOPS-PEDOT:PSS', 'PEDOT', 'PEDOT-Cl', 'PEDOT-MoS2', 'PEDOT-P',
       'PEDOT-PBA', 'PEDOT-T', 'PEDOT-b-PEG',
       'PEDOT-cobalt(II) hexylridge', 'PEDOT-silver perchlorate',
       'PEDOT:PSS', 'PEDOT:PSS.', 'PEDOT:PSS/SPEEK', 'PEDOT:PolyDADMA',
       'PEDOT:PolyDADMA FSI', 'PEDOT:PolyDADMA TFSI', 'PEDOT:Tos',
       'PEO-PEDOT', 'МIP-PEDOT-PBA'], dtype='<U27')

In [262]:
df_polímeros.to_excel('1_OMIEC_RESPONSES_output_preprocessed.xlsx', index=False)

In [ ]:
df